<a href="https://colab.research.google.com/github/anushayerram2025/AIML/blob/main/Copy_of_AIML_Module_3_Lab_3_Using_KNN_for_Text_Classification_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification



---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [14]:
"""
Stemming involves reducing words to their base or root form by removing suffixes.
Original: "running, runner, ran"
Stemmed: "run, run, ran"

Lemmatization involves reducing words to their base or root form by considering the meaning of the word and its context.
Original: "running, runner, ran"
Lemmatized: "run, runner, run"

Stemming tends to be faster and simpler, but it may not always produce a valid word or the actual root form.
Lemmatization is more sophisticated, considering the context and meaning of words, but it can be slower and computationally more intensive.
"""

'\nStemming involves reducing words to their base or root form by removing suffixes.\nOriginal: "running, runner, ran"\nStemmed: "run, run, ran"\n\nLemmatization involves reducing words to their base or root form by considering the meaning of the word and its context.\nOriginal: "running, runner, ran"\nLemmatized: "run, runner, run"\n\nStemming tends to be faster and simpler, but it may not always produce a valid word or the actual root form.\nLemmatization is more sophisticated, considering the context and meaning of words, but it can be slower and computationally more intensive.\n'

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [16]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [3]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [5]:
5*12

60

In [6]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    # If remove_stopwords is True, create a CountVectorizer with English stop words.
    # If remove_stopwords is False, create a CountVectorizer without removing stop words.
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    # Clean the training data by applying the cleanText function to each paragraph.
    # The cleanText function takes care of lemmatization and stemming based on the provided parameters.
    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    # Clean the test data in a similar way.
    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    # Use the CountVectorizer to transform the cleaned training and test data into Bag of Words representations.
    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()

    # Return the Bag of Words representations for training and test datasets.
    return bag_of_words_train, bag_of_words_test



In [21]:
"""Bag of words:
Eg:
"Anusha Loves coding."
"Anusha loves cooking"
Step 1: Tokenization
Break each sentence into individual words. This step involves removing punctuation and splitting the text into a list of words.
["Anusha","Loves","coding"]
["Anusha","studies","cooking"]

Step 2: Vocabulary Building
Create a vocabulary by collecting all unique words from the tokenized sentences.

Vocabulary: ["Anusha","loves","coding","cooking"]

Step 3: Create Vectors
Represent each sentence as a vector in the space of the vocabulary. The values in the vector correspond to the frequency of each word in the sentence.
[1,1,1,0]
[1,1,0,1]"""

'Bag of words:\nEg:\n"Anusha Loves coding."\n"Anusha loves cooking"\nStep 1: Tokenization\nBreak each sentence into individual words. This step involves removing punctuation and splitting the text into a list of words.\n["Anusha","Loves","coding"]\n["Anusha","studies","cooking"]\n\nStep 2: Vocabulary Building\nCreate a vocabulary by collecting all unique words from the tokenized sentences.\n\nVocabulary: ["Anusha","loves","coding","cooking"]\n\nStep 3: Create Vectors\nRepresent each sentence as a vector in the space of the vocabulary. The values in the vector correspond to the frequency of each word in the sentence.\n[1,1,1,0]\n[1,1,0,1]'

## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [22]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [9]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [10]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [11]:
df = df.dropna()

In [12]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


In [13]:
df.to_csv('reviews.csv', index=False)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [30]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [31]:
predicted, y_test = bow_knn()

<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 62.30366492146597%
Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]




/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [34]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.73
[0.7254902  0.74117647 0.72834646]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [32]:
#TASK 1 ka solution by modifying different parameters...using a list parameter_sets_bow
def bow_knn1():
    # List of dictionaries containing different sets of parameters for BoW
    parameter_sets_bow = [
        {'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'auto', 'metric': 'euclidean'},
        {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'ball_tree', 'metric': 'manhattan'},
    ]

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)

    for params in parameter_sets_bow:
        print(f"\nExperimenting with parameters: {params}")

        X_train_bow, X_test_bow = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

        knn = neighbors.KNeighborsClassifier(**params)
        knn.fit(X_train_bow, y_train)
        predicted = knn.predict(X_test_bow)

        acc = metrics.accuracy_score(y_test, predicted)
        print(f'KNN with BOW accuracy = {acc * 100}%')

        scores = cross_val_score(knn, X_train_bow, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
        print(scores)

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [33]:
## KNN accuracy after using BoW
bow_knn1()


Experimenting with parameters: {'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'auto', 'metric': 'euclidean'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 59.16230366492147%
Cross Validation Accuracy: 0.59

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



[0.54901961 0.60784314 0.62204724]

Experimenting with parameters: {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'ball_tree', 'metric': 'manhattan'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 68.58638743455498%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.63
[0.62352941 0.59215686 0.67716535]


# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [36]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [37]:
import pandas as pd
df = pd.read_csv('spam.csv', error_bad_lines=False)
df

<ipython-input-37-3b52591eaf2b>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('spam.csv', error_bad_lines=False)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [38]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [39]:
len(df)

5572

In [40]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [ ]:
# This cell may take some time to run
predicted, y_test = bow_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]




In [ ]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.97
[0.96837147 0.96769852 0.96363636]


In [43]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score


def bow_knn_2():
    """Experiment with different parameters and distance metrics for BoW and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)

    # List of dictionaries containing different sets of parameters for BoW
    parameter_sets_bow = [
        {'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'auto', 'metric': 'euclidean'},
        {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto', 'metric': 'manhattan'}
    ]

    for params in parameter_sets_bow:
        print(f"\nExperimenting with BoW parameters: {params}")

        X_train_bow, X_test_bow = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

        knn = neighbors.KNeighborsClassifier(**params)
        knn.fit(X_train_bow, y_train)
        predicted = knn.predict(X_test_bow)

        acc = metrics.accuracy_score(y_test, predicted)
        print(f'KNN with BoW accuracy = {acc * 100}%')

        scores = cross_val_score(knn, X_train_bow, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
        print(scores)


def tfidf_knn_2():
    """Experiment with different parameters and distance metrics for TF-IDF and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)

    # List of dictionaries containing different sets of parameters for TF-IDF
    parameter_sets_tfidf = [
        {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'cosine'},
        {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'brute', 'metric': 'euclidean'},
        {'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'auto', 'metric': 'manhattan'}
    ]

    for params in parameter_sets_tfidf:
        print(f"\nExperimenting with TF-IDF parameters: {params}")

        X_train_tfidf, X_test_tfidf = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

        knn = neighbors.KNeighborsClassifier(**params)
        knn.fit(X_train_tfidf, y_train)
        predicted = knn.predict(X_test_tfidf)

        acc = metrics.accuracy_score(y_test, predicted)
        print(f'KNN with TF-IDF accuracy = {acc * 100}%')

        scores = cross_val_score(knn, X_train_tfidf, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
        print(scores)

# Run experiments



In [44]:
bow_knn_2()


Experimenting with BoW parameters: {'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'auto', 'metric': 'euclidean'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BoW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]

Experimenting with BoW parameters: {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto', 'metric': 'manhattan'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BoW accuracy = 92.91479820627802%
Cross Validation Accuracy: 0.92
[0.92328398 0.9179004  0.92255892]


In [47]:

tfidf_knn_2()


Experimenting with TF-IDF parameters: {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'cosine'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TF-IDF accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.97
[0.96837147 0.96769852 0.96363636]

Experimenting with TF-IDF parameters: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'brute', 'metric': 'euclidean'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TF-IDF accuracy = 88.96860986547085%
Cross Validation Accuracy: 0.88
[0.87617766 0.87550471 0.88080808]

Experimenting with TF-IDF parameters: {'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'auto', 'metric': 'manhattan'}


<ipython-input-16-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TF-IDF accuracy = 92.10762331838565%
Cross Validation Accuracy: 0.91
[0.91520861 0.91117093 0.91582492]


### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

In [46]:
"""
1) TF-idf not only focusses on frequency of words just like bag of words but rather it focusses on importance in the sentence also.
2)Word2Vec is one of the techniques better than both BoW and TF-IDF
3)pros:Stemming reduces words to root or base form. whereas lemmatization retains the base and also preserves the semantic meaning.
cons:stemming leads to loss of meaning of the word. Lemmatization is costly and not be used everytime.

"""

'\n1) TF-idf not only focusses on frequency of words just like bag of words but rather it focusses on importance in the sentence also.\n2)Word2Vec is one of the techniques better than both BoW and TF-IDF\n3)pros:Stemming reduces words to root or base form. whereas lemmatization retains the base and also preserves the semantic meaning.\ncons:stemming leads to loss of meaning of the word. Lemmatization is costly and not be used everytime.\n\n'

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
